In [1]:
from herbie import Herbie


In [2]:
H = Herbie("2021-07-23", model="rrfs", fxx=1, member=1)


✅ Found ┊ model=rrfs ┊ product=mean ┊ 2021-Jul-23 00:00 UTC F01 ┊ GRIB2 @ aws ┊ IDX @ aws


In [3]:
H.SOURCES


{'aws': 'https://noaa-rrfs-pds.s3.amazonaws.com/rrfs.20210723/00/ensprod/rrfsce.t00z.conus.mean.f01.grib2',
 'aws-mem': 'https://noaa-rrfs-pds.s3.amazonaws.com/rrfs.20210723/00/mem01/rrfs.t00z.mem01.meanf001.grib2'}

In [4]:
H.PRODUCTS


{'mean': 'ensemble mean',
 'avrg': 'ensemble products: ???',
 'eas': 'ensemble products: ???',
 'ffri': 'ensemble products: ???',
 'lpmm': 'ensemble products: ???',
 'pmmn': 'ensemble products: ???',
 'prob': 'ensemble products: ???',
 'testbed.conus': 'surface grids (one for each member)',
 'na': 'native grids (one for each member)'}

In [5]:
ds = H.xarray("TMP:2 m")
ds

👨🏻‍🏭 Created directory: [/Users/johnlawson/data/rrfs/20210723/mem01]


<xarray.Dataset>
Dimensions:              (y: 1059, x: 1799)
Coordinates:
    time                 datetime64[ns] 2021-07-23
    step                 timedelta64[ns] 01:00:00
    heightAboveGround    float64 2.0
    latitude             (y, x) float64 21.14 21.14 21.15 ... 47.86 47.85 47.84
    longitude            (y, x) float64 237.3 237.3 237.3 ... 299.0 299.0 299.1
    valid_time           datetime64[ns] 2021-07-23T01:00:00
Dimensions without coordinates: y, x
Data variables:
    t2m                  (y, x) float32 295.3 295.3 295.3 ... 287.7 287.8 287.8
    gribfile_projection  object None
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          2
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   rrfs
    product:                 mean
    description:             Rapid Refresh Forecast System (RRFS) Ensemble
    remote_grib:             https://noaa-rrfs-pds.s3.amazonaws.com/rrfs.2021...
    local_grib:              /Users/johnlawson/data/rrfs/20210723/mem01/subse...
    searchString:            TMP:2 m

In [6]:
# Missing index files?
H = Herbie(
    "2021-07-23", model="rrfs", product="testbed.conus", member=1, priority="aws-mem"
)

✅ Found ┊ model=rrfs ┊ product=testbed.conus ┊ 2021-Jul-23 00:00 UTC F00 ┊ GRIB2 @ aws-mem ┊ IDX @ aws-mem


In [7]:
# no index file available. Have to download the full file.
H.download()

PosixPath('/Users/johnlawson/data/rrfs/20210723/mem01/rrfsce.t00z.conus.testbed.conus.f00.grib2')

In [8]:
import xarray
# Since we have the full file, we need to filter by keys to open the
# variable we want with cfgrib
x = xarray.open_dataset(
    H.get_localFilePath(),
    engine="cfgrib",
    backend_kwargs={
        "filter_by_keys": {"shortName": "2t", "typeOfLevel": "heightAboveGround"}
    },
)

In [9]:
import metpy
crs_info = x.metpy.parse_cf().metpy_crs.item()
crs_info.to_cartopy()
crs_info.to_pyproj()


/var/folders/rr/m6rsyqmd64q6ddlcqfq1sng00000gp/T/ipykernel_48106/1094693504.py:2: UserWarning: More than one time coordinate present for variable  "t2m".
  crs_info = x.metpy.parse_cf().metpy_crs.item()


AttributeError: 'Dataset' object has no attribute 'metpy_crs'